In [1]:
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 15.2 MB/s eta 0:00:00


In [39]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
import tensorflow_io as tfio
from IPython.display import Audio

filename = '00.mp3'

#частота дискритизации для работы
rate_out=8000

audio = tfio.audio.AudioIOTensor(filename)
print(audio)

audio_len = int(audio.shape[0])
# частота дискритизации исходника
rate = int(audio.rate)
chank = 1000000

audio_f = audio[chank: audio_len-chank]  # Обрежем тишину
wav = audio_f[:,0] # Берем только первый канал звука
wav = tfio.audio.resample(wav, rate_in=rate, rate_out=rate_out)

print(wav)

#Audio(wav, rate=rate_out)



<AudioIOTensor: shape=[66003264        2], dtype=<dtype: 'float32'>, rate=44100>
tf.Tensor(
[ 2.3139691e-07 -8.6940059e-07  3.0053519e-07 ...  1.2727627e-01
  7.2145529e-02  8.1847675e-02], shape=(11610569,), dtype=float32)


In [169]:
# Нарежем звук на части по 0,1 сек
smpl = wav.shape[0] // 800
len800 = 800*smpl
wav800 = wav[:len800]
t2 = tf.reshape(wav800, [-1, 800])
tf.shape(t2)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([14513,   800], dtype=int32)>

In [171]:
# Разделим данные
len_train = int(0.9 * smpl)
x_train = t2[:len_train]
x_test = t2[len_train:]
x_train.shape

In [205]:
model = tf.keras.Sequential()
# Добавим слой Embedding ожидая на входе словарь размера 800,
# и на выходе вложение размерностью 800.
model.add(layers.Input([800,1]))
model.add(layers.GRU(256))
model.add(layers.Dense(256))
model.add(layers.Dense(10))
model.add(layers.Dense(256))
model.add(layers.Dense(800))
model.summary()
model.compile(optimizer='adamax', loss='mean_absolute_error', metrics=['accuracy'])
model.fit(x_train, x_train, batch_size=64, epochs=50)


Model: "sequential_112"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_34 (GRU)                (None, 800)               1927200   
                                                                 
 dense_434 (Dense)           (None, 256)               205056    
                                                                 
 dense_435 (Dense)           (None, 10)                2570      
                                                                 
 dense_436 (Dense)           (None, 256)               2816      
                                                                 
 dense_437 (Dense)           (None, 800)               205600    
                                                                 
Total params: 2343242 (8.94 MB)
Trainable params: 2343242 (8.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
 19

KeyboardInterrupt: 

In [ ]:
print(model.weights )